<a href="https://colab.research.google.com/github/estefaniabrito/SYS/blob/main/PRUEBA_DEL_AUDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from google.colab import drive
import os
import zipfile
import joblib


drive.mount('/content/drive')


zip_path = "/content/drive/MyDrive/2025_02_09_00_06_modelo.zip"
extract_path = "/content/modelo_extraido"


if not os.path.exists(extract_path):
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

modelo_pkl = os.path.join(extract_path, "deteccion_palabra_clave.pkl")


modelo = joblib.load(modelo_pkl)
print("Modelo cargado con éxito:", modelo_pkl)
print(" Claves del modelo:", modelo.keys())

Xw_ = modelo["Xw_"]  # Espectros normalizados
fmax = modelo["fmax"]  # Frecuencia máxima considerada
labels = modelo["labels"]  # Etiquetas (0 = sin clave, 1 = con clave)
vf = modelo["vf"]  # Vector de frecuencias
fs = modelo["fs"]  # Frecuencia de muestreo


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modelo cargado con éxito: /content/modelo_extraido/deteccion_palabra_clave.pkl
 Claves del modelo: dict_keys(['Xw_', 'fmax', 'labels', 'vf', 'fs'])


In [63]:
import librosa
import soundfile as sf
import numpy as np

# Ruta del archivo de audio a analizar (puede no ser WAV)
input_audio = "/content/drive/MyDrive/AUDIOS DE PRUEBA/PTT-20250208-WA0044.opus"  # Cambia por tu archivo

#  Convertir a WAV si es necesario
output_audio = "audio_prueba.wav"
x, fs = librosa.load(input_audio, sr=fs, mono=False)

# Si el audio es mono, convertir a estéreo
if x.ndim == 1:
    x = np.stack([x, x], axis=-1)

# Guardar el archivo en WAV para análisis
sf.write(output_audio, x, fs)
print(f" Audio convertido a WAV: {output_audio}, {fs} Hz, {x.shape[1]} canales")


 Audio convertido a WAV: audio_prueba.wav, 48000 Hz, 2 canales


In [64]:
#  Definir la duración de cada segmento
ts = 1  # Segundos por segmento
muestras_segmento = int(ts * fs)  # Muestras por cada segmento
duracion_total = x.shape[0] / fs  # Duración total en segundos
tl = np.arange(0, duracion_total, ts)  # Tiempos de inicio de cada segmento

#  Crear matriz para almacenar los segmentos
Ns = len(tl)  # Número de segmentos
x_t = np.zeros((Ns, muestras_segmento, x.shape[1]))  # Matriz para los segmentos

#  Extraer segmentos asegurando que todos tengan la misma dimensión
for i, ti in enumerate(tl):
    start = int(fs * ti)
    end = start + muestras_segmento

    # Si el segmento es más corto, rellenamos con ceros
    if end > x.shape[0]:
        segmento = np.zeros((muestras_segmento, x.shape[1]))
        segmento[: x.shape[0] - start, :] = x[start:, :]
    else:
        segmento = x[start:end, :]

    x_t[i] = segmento  # Guardar el segmento en la matriz

print(f" Segmentación completada: {x_t.shape}")


 Segmentación completada: (3, 48000, 2)


In [65]:
from sklearn.preprocessing import MinMaxScaler

#  Calcular Transformada de Fourier y normalizar espectros
Xw = np.fft.rfft(x_t, axis=1).mean(axis=-1)  # FFT y promediado de canales
sca = MinMaxScaler()
Xw_entrada = sca.fit_transform(np.abs(Xw).T).T  # Normalización

print(" FFT calculada y espectros normalizados:", Xw_entrada.shape)


 FFT calculada y espectros normalizados: (3, 24001)


In [66]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#  Calcular Transformada de Fourier y normalizar espectros
Xw = np.fft.rfft(x_t, axis=1).mean(axis=-1)  # FFT y promediado de canales
sca = MinMaxScaler()
Xw_entrada = sca.fit_transform(np.abs(Xw).T).T  # Normalización

print(" FFT calculada y espectros normalizados:", Xw_entrada.shape)

# Asegurar que Xw_ tenga la misma forma que Xw_entrada
# Supongamos que Xw_ es el espectro de referencia (ajusta según tu caso)
Xw_ = np.random.rand(3, 24001)  # Ejemplo: misma forma que Xw_entrada

#  Calcular la distancia entre los espectros del modelo y los nuevos
distancias = np.linalg.norm(Xw_entrada - Xw_, axis=1)

#  Definir un umbral para detección (ajustar según pruebas)
umbral = np.percentile(distancias, 30)  # 30% de los valores más bajos

#  Predecir si cada segmento tiene la palabra clave
detecciones = distancias < umbral  # True = detectado, False = no detectado

#  Mostrar resultados
for i, detectado in enumerate(detecciones):
    print(f" Segmento {i} ({tl[i]}s - {tl[i]+ts}s): {'NO' if not detectado else ' SÍ'} contiene la palabra clave")

#  Mostrar cuántos segmentos detectaron la palabra clave
print(f"\n Total segmentos con palabra clave: {sum(detecciones)} de {len(detecciones)}")

 FFT calculada y espectros normalizados: (3, 24001)
 Segmento 0 (0.0s - 1.0s):  SÍ contiene la palabra clave
 Segmento 1 (1.0s - 2.0s): NO contiene la palabra clave
 Segmento 2 (2.0s - 3.0s): NO contiene la palabra clave

 Total segmentos con palabra clave: 1 de 3


In [67]:
import numpy as np
from IPython.display import Audio

#  Reproducir solo un segmento donde se detectó la palabra clave
segmentos_detectados = np.where(detecciones)[0]  # Índices de los segmentos detectados

if len(segmentos_detectados) > 0:
    i = segmentos_detectados[0]  # Tomar el primer segmento detectado
    print(f" Reproduciendo segmento {i} con la palabra clave:")
    print(f"Tiempo: {tl[i]}s - {tl[i]+ts}s")  # Mostrar el tiempo del segmento
    Audio(x_t[i].T, rate=fs)  # Reproducir el segmento
else:
    print(" No se detectó la palabra clave en ningún segmento.")

 Reproduciendo segmento 0 con la palabra clave:
Tiempo: 0.0s - 1.0s


In [68]:
#  Definir la duración de cada segmento
ts = 1  # Segundos por segmento
muestras_segmento = int(ts * fs)  # Muestras por cada segmento
duracion_total = x.shape[0] / fs  # Duración total en segundos
tl = np.arange(0, duracion_total, ts)  # Tiempos de inicio de cada segmento

#  Crear matriz para almacenar los segmentos
Ns = len(tl)  # Número de segmentos
x_t = np.zeros((Ns, muestras_segmento, x.shape[1]))  # Matriz para los segmentos

#  Extraer segmentos asegurando que todos tengan la misma dimensión
for i, ti in enumerate(tl):
    start = int(fs * ti)
    end = start + muestras_segmento

    # Si el segmento es más corto, rellenamos con ceros
    if end > x.shape[0]:
        segmento = np.zeros((muestras_segmento, x.shape[1]))
        segmento[: x.shape[0] - start, :] = x[start:, :]
    else:
        segmento = x[start:end, :]

    x_t[i] = segmento  # Guardar el segmento en la matriz

print(f" Segmentación completada: {x_t.shape}")

#  Calcular Transformada de Fourier y normalizar espectros
Xw = np.fft.rfft(x_t, axis=1).mean(axis=-1)  # FFT y promediado de canales
sca = MinMaxScaler()
Xw_entrada = sca.fit_transform(np.abs(Xw).T).T  # Normalización

print(" FFT calculada y espectros normalizados:", Xw_entrada.shape)

#  Calcular la distancia entre los espectros del modelo y los nuevos
distancias = np.linalg.norm(Xw_entrada - Xw_, axis=1)

#  Definir un umbral para detección (ajustar según pruebas)
umbral = np.percentile(distancias, 30)  # 30% de los valores más bajos

#  Predecir si cada segmento tiene la palabra clave
detecciones = distancias < umbral  # True = detectado, False = no detectado

#  Mostrar resultados
for i, detectado in enumerate(detecciones):
    print(f" Segmento {i} ({tl[i]}s - {tl[i]+ts}s): {' NO' if not detectado else ' SÍ'} contiene la palabra clave")

#  Mostrar cuántos segmentos detectaron la palabra clave
print(f"\n Total segmentos con palabra clave: {sum(detecciones)} de {len(detecciones)}")

#  Reproducir solo un segmento donde se detectó la palabra clave
segmentos_detectados = np.where(detecciones)[0]  # Índices de los segmentos detectados

if len(segmentos_detectados) > 0:
    i = segmentos_detectados[0]  # Tomar el primer segmento detectado
    print(f"\n Reproduciendo segmento {i} con la palabra clave:")
    print(f"Tiempo: {tl[i]}s - {tl[i]+ts}s")  # Mostrar el tiempo del segmento
    Audio(x_t[i].T, rate=fs)  # Reproducir el segmento
else:
    print("\n No se detectó la palabra clave en ningún segmento.")

 Segmentación completada: (3, 48000, 2)
 FFT calculada y espectros normalizados: (3, 24001)
 Segmento 0 (0.0s - 1.0s):  SÍ contiene la palabra clave
 Segmento 1 (1.0s - 2.0s):  NO contiene la palabra clave
 Segmento 2 (2.0s - 3.0s):  NO contiene la palabra clave

 Total segmentos con palabra clave: 1 de 3

 Reproduciendo segmento 0 con la palabra clave:
Tiempo: 0.0s - 1.0s
